# Quick Guide

- author  : ZL
- datetime: 20200915
- link    : https://docs.python.org/3/library/unittest.mock.html

Mock and MagicMock objects create all attributes and methods as you access them and store details of how they have been used. 

You can configure them, to specify return values or limit what attributes are available, and then make assertions about how they have been used:

In [ ]:
from unittest.mock import MagicMock

class ProductionClass:
    pass

In [10]:
thing = ProductionClass()
thing.method = MagicMock(return_value=1)

In [11]:
thing.method(3, 4, 5, key='value')

1

In [12]:
thing.method.assert_called_with(3, 4, 5, key='value')

## side_effect

allows you to perform side effects,
 
including raising an exception when a mock is called:

In [6]:
from unittest.mock import Mock
mock = Mock(side_effect=KeyError('foo'))
mock()

KeyError: 'foo'

In [7]:
values = dict(zip('abc', range(1,4)))
def side_effect(arg):
    return values[arg]

In [9]:
mock.side_effect = side_effect

In [10]:
mock('a'), mock('b'), mock('c')

(1, 2, 3)

In [11]:
mock.side_effect = range(5, 0, -1)

In [12]:
mock(), mock(), mock()

(5, 4, 3)

## Configure mock and control its behavior

Mock has many other ways you can configure it and control its behavior.

for example, the <font color="blue">spec</font> argument configures the mock to take its specification from another object.

attempting to access attributes or methods on the mock that don't exist on the spec will fail with an <font color="blue">AttributeError</font>

## patch()

the patch() decorator / context manager makes it easy to mock classes or objects in a module under test.

the object you specify will be replaced with a mock ( or other object)
during the test and restored when the test ends.

In [18]:
from unittest.mock import patch
import module

In [19]:
@patch('module.ClassName2')
@patch('module.ClassName1')
def test_module_classes(MockClass1, MockClass2):
    module.ClassName1()
    module.ClassName2()
    assert MockClass1 is module.ClassName1
    assert MockClass2 is module.ClassName2
    assert MockClass1.called
    assert MockClass2.called

In [20]:
test_module_classes()

### NOTE:

when you nest patch decorators, the mocks are passed into the decorated function in the same order they applied (the normal order that decorator are applied.)

this means from the bottom up, so in the example above the mock for mock.ClassName1 is passed in first

## patch.dict()

there is also patch.dict() for setting values in dictionary just during a scope and restoring the dictionary to its original state when the test ends.

In [21]:
foo = {'key': 'value'}
original = foo.copy()

with patch.dict(foo, {'newkey': 'newvalue'}, clear=True):
    # should pass
    assert foo == {'newkey': 'newvalue'}

In [22]:
assert foo == original # should pass

## Mock python data-model methods

mock supports the mocking of python magic methods.

the easiest way of using magic methods is with the MagicMock class.

it allows you to do things like:

In [23]:
mock = MagicMock()
mock.__str__.return_value = 'foobarbaz'
str(mock)

'foobarbaz'

In [24]:
mock.__str__.assert_called_with()

Mock allows you to assign functions (or other Mock instances) to magic methods and they will be called appropriately.

<font color="red">the MagicMock class is just a Mock variant that has all of the magic methods pre-created for you (well, all the useful ones anyway)</font>

the following is an example of using magic methods with the ordinary Mock class:

In [25]:
mock = Mock()
mock.__str__ = Mock(return_value='weeeeeeee') # this line is different
str(mock)

'weeeeeeee'

In [26]:
mock.__str__.assert_called_with()

## auto-speccing

For ensuring that the mock obj in tests have the same API as the obj they are replacing, you can use <font color="blue">auto-speccing</font>.

auto-speccing can be done throu the "autospec" argument to patch, or the "create_autospec()" function.

auto-speccing creates mock obj that have the same attributes and methods as the obj they are replacing, and any functions and methods (including constructors) have the same call signature as the real obj.

this ensures that your mocks will fail in the same way as your production code if they are used incorrectly.

In [28]:
from unittest.mock import create_autospec
def func(a, b, c):
    pass

In [30]:
mock_function = create_autospec(func, return_value='fishy')
mock_function(1, 2, 3)

'fishy'

In [31]:
mock_function.assert_called_once_with(1, 2, 3)

In [32]:
mock_function('wrong argument') # missed 2 position args, Exception

TypeError: missing a required argument: 'b'

# The Mock Class

- Mock is a flexible mock obj intended to replace the use of stubs and test doubles throughout your code.

Mocks are callable and create attributes as new mocks when you access them.

accessing the same attributes will always return the same mock.

Mocks record how you use them, allowing you to mock assertions about what your code has done to them.

- MagicMock is a subclass of Mock with all the magic methods pre-created and ready to use.

There are also non-callable variants, useful when you are mocking out objects that aren't callable: "NonCallableMock" and "NonCallableMagic"

- the patch() decorators make it easy to temporarily replace classes in a particular module with a Mock object.

by default, patch() will create a MagicMock for you. You can specify an alternative class of Mock using the new_callable argument to patch().

## assert_called()

In [33]:
mock = Mock()
mock.method()

<Mock name='mock.method()' id='128789168'>

In [34]:
mock.method.assert_called()

## assert_called_once()

In [35]:
mock = Mock()
mock.method()

<Mock name='mock.method()' id='132850800'>

In [36]:
mock.method.assert_called_once()

In [37]:
mock.method()

<Mock name='mock.method()' id='132850800'>

In [38]:
mock.method.assert_called_once() # exception

AssertionError: Expected 'method' to have been called once. Called 2 times.

## assert_called_with(*args, **kwargs)

this method is a convient way of asserting that the last call has been made in a particular way.

In [39]:
mock = Mock()
mock.method(1, 2, 3, test='reee')

<Mock name='mock.method()' id='129532784'>

In [40]:
mock.method.assert_called_with(1, 2, 3, test='reee')

## assert_called_once_with(*args, **kwargs)

combination of assert_called_with() and assert_called_once()

In [41]:
mock = Mock(return_value=None)
mock('foo', bar='baz')
mock.assert_called_once_with('foo', bar='baz') # pass
mock('other', bar='values')
mock.assert_called_once_with('other', bar='values') # fail

AssertionError: Expected 'mock' to be called once. Called 2 times.

## assert_any_call(*args, **kwargs)

assert the mock has been called with the specified arguments.

the assert passes if the mock has ever been called.

unlike assert_called_with() and assert_called_once_with() that only pass if the call is the most recent one, and in the case of assert_called_once_with() it must also be the only call.

In [42]:
mock = Mock(return_value=None)
mock(1, 2, arg='thing')
mock('some', 'thing', 'else')
mock.assert_any_call(1, 2, arg='thing')

## assert_has_calls(calls, any_order=False)

assert the mock has been called with the specified calls. The mock_calls list is checked for the calls.

If any_order is false then the calls must be sequential. There can be extra calls before or after the specified calls.

If any_order is true then the calls can be in any order, but they must all appear in mock_calls.


In [45]:
from unittest.mock import call
mock = Mock(return_value=None)
mock(1)
mock(2)
mock(3)
mock(4)
calls = [call(2), call(3)]
mock.assert_has_calls(calls)
calls = [call(4), call(2), call(3)]
mock.assert_has_calls(calls, any_order=True) # pass? nah, have to import all

## assert_not_called()

Assert the mock was never called.

In [46]:
m = Mock()
m.hello.assert_not_called()

In [47]:
obj = m.hello() # called once
m.hello.assert_not_called() # fail

AssertionError: Expected 'hello' to not have been called. Called 1 times.

## reset_mock()


The reset_mock method resets all the call attributes on a mock object.

This can be useful where you want to make a series of assertions that reuse the same object. 

In [48]:
mock = Mock(return_value=None)
mock('hello')
mock.called

True

In [49]:
mock.reset_mock()

In [51]:
mock.called

False

## mock_add_spec(spec, spec_set=False)

Add a spec to a mock. spec can either be an object or a list of strings. Only attributes on the spec can be fetched as attributes from the mock.

If spec_set is true then only attributes on the spec can be set.

## attach_mock(mock, attribute)

Attach a mock as an attribute of this one, replacing its name and parent. Calls to the attached mock will be recorded in the method_calls and mock_calls attributes of this one.